In [44]:
import pandas as pd
import sqlite3

In [45]:
df_cleaned = pd.read_csv("walmart_clean_data.csv")

In [46]:
df_cleaned.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48,604.17


In [47]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9969 entries, 0 to 9968
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      9969 non-null   int64  
 1   Branch          9969 non-null   object 
 2   City            9969 non-null   object 
 3   category        9969 non-null   object 
 4   unit_price      9969 non-null   float64
 5   quantity        9969 non-null   float64
 6   date            9969 non-null   object 
 7   time            9969 non-null   object 
 8   payment_method  9969 non-null   object 
 9   rating          9969 non-null   float64
 10  profit_margin   9969 non-null   float64
 11  total           9969 non-null   float64
dtypes: float64(5), int64(1), object(6)
memory usage: 934.7+ KB


In [48]:
conn = sqlite3.connect("walmart_sales.db")

In [49]:
df_cleaned.to_sql("walmart", conn, if_exists="replace", index=False)

9969

In [50]:
query = """
SELECT *
FROM walmart

"""
pd.read_sql_query(query, conn)


,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total
0,1,WALM003,San Antonio,Health and beauty,74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48,604.17
...,...,...,...,...,...,...,...,...,...,...,...,...
9964,9996,WALM056,Rowlett,Fashion accessories,37.00,3.0,03/08/23,10:10:00,Cash,3.0,0.33,111.00
9965,9997,WALM030,Richardson,Home and lifestyle,58.00,2.0,22/02/21,14:20:00,Cash,7.0,0.48,116.00
9966,9998,WALM050,Victoria,Fashion accessories,52.00,3.0,15/06/23,16:00:00,Credit card,4.0,0.48,156.00
9967,9999,WALM032,Tyler,Home and lifestyle,79.00,2.0,25/02/21,12:25:00,Cash,7.0,0.48,158.00


In [56]:
df_cleaned['unit_price'] = df_cleaned['unit_price'].replace(r'[\$,]', '', regex=True).astype(float)


In [57]:
query = """
SELECT Branch, 
       ROUND(SUM(unit_price * quantity), 2) AS total_revenue
FROM walmart
GROUP BY Branch
ORDER BY total_revenue DESC;
"""
pd.read_sql_query(query, conn)


,Branch,total_revenue
0,WALM009,25688.34
1,WALM074,25555.42
2,WALM003,24950.56
3,WALM058,24524.37
4,WALM030,24460.60
...,...,...
95,WALM039,6769.33
96,WALM033,6572.91
97,WALM013,6237.11
98,WALM031,5568.84


In [58]:
query = """
SELECT Branch, 
       ROUND(AVG(unit_price * quantity), 2) AS avg_transaction_value
FROM walmart
GROUP BY Branch
ORDER BY avg_transaction_value DESC;
"""
pd.read_sql_query(query, conn)



,Branch,avg_transaction_value
0,WALM017,182.73
1,WALM042,174.06
2,WALM062,163.23
3,WALM066,162.67
4,WALM095,162.62
...,...,...
95,WALM036,105.32
96,WALM048,102.92
97,WALM058,102.61
98,WALM031,99.44


In [59]:
query = """
SELECT category,
       ROUND(SUM(unit_price * quantity * profit_margin), 2) AS estimated_profit
FROM walmart
GROUP BY category
ORDER BY estimated_profit DESC;
"""
pd.read_sql_query(query, conn)

,category,estimated_profit
0,Fashion accessories,192314.89
1,Home and lifestyle,192213.64
2,Electronic accessories,30772.49
3,Food and beverages,21552.86
4,Sports and travel,20613.81
5,Health and beauty,18671.73


In [60]:
query = """
SELECT strftime('%H', time) AS hour_of_day, COUNT(*) AS total_transactions
FROM walmart
GROUP BY hour_of_day
ORDER BY total_transactions DESC;
"""
pd.read_sql_query(query, conn)

,hour_of_day,total_transactions
0,None,1273
1,15,1191
2,16,1173
3,17,1027
4,19,1024
5,18,986
6,20,972
7,13,436
8,10,411
9,12,409


In [62]:
query = """
SELECT branch, city, ROUND(SUM(unit_price * quantity), 2) AS revenue
FROM walmart
GROUP BY branch, city
ORDER BY revenue DESC;
"""
pd.read_sql_query(query, conn)

,Branch,City,revenue
0,WALM009,Plano,25688.34
1,WALM074,Weslaco,25555.42
2,WALM003,San Antonio,24950.56
3,WALM058,Port Arthur,24524.37
4,WALM030,Richardson,24460.60
...,...,...,...
95,WALM039,Longview,6769.33
96,WALM033,Pearland,6572.91
97,WALM013,Irving,6237.11
98,WALM031,Lewisville,5568.84


In [64]:
query = """
SELECT category, SUM(quantity) AS total_units_sold
FROM walmart
GROUP BY category
ORDER BY total_units_sold DESC
LIMIT 5;
"""
pd.read_sql_query(query, conn)

,category,total_units_sold
0,Fashion accessories,9653.0
1,Home and lifestyle,9610.0
2,Electronic accessories,1494.0
3,Food and beverages,952.0
4,Sports and travel,920.0
